# 51. Introduction to Cumulative Aggregations and Ranking in SQL Queries

# 52. Overview of CTAS to create tables based on Query Results

In [1]:
%%sql
CREATE TABLE order_count_by_status
AS
SELECT orders.order_status, count(*) AS order_count
FROM orders
GROUP BY 1;

SELECT * FROM order_count_by_status;

CREATE TABLE orders_stg
AS
SELECT * FROM orders WHERE 1=2;

-- SELECT o.order_date,
--     round(sum(oi.order_item_subtotal)::numeric, 2) AS order_revenue
-- FROM orders AS o
--     JOIN order_items AS oi
--         ON o.order_id = oi.order_item_order_id
-- WHERE o.order_status IN ('COMPLETE', 'CLOSED')
-- GROUP BY 1

,order_status,order_count
0,COMPLETE,22899
1,ON_HOLD,3798
2,PENDING_PAYMENT,15030
3,PENDING,7610
4,CLOSED,7556
5,CANCELED,1428
6,PROCESSING,8275
7,PAYMENT_REVIEW,729
8,SUSPECTED_FRAUD,1558


In [2]:
%%sql
SELECT * FROM orders_stg;

,order_id,order_date,order_customer_id,order_status


In [3]:
%%sql
SELECT * FROM order_count_by_status;

,order_status,order_count
0,COMPLETE,22899
1,ON_HOLD,3798
2,PENDING_PAYMENT,15030
3,PENDING,7610
4,CLOSED,7556
5,CANCELED,1428
6,PROCESSING,8275
7,PAYMENT_REVIEW,729
8,SUSPECTED_FRAUD,1558


# 53. Create Tables for Cumulative Aggregations and Ranking

In [ ]:
%%sql
CREATE TABLE daily_revenue
AS
SELECT o.order_date,
       ROUND(SUM(oi.order_item_subtotal)::NUMERIC, 2) AS order_revenue
FROM orders AS o
JOIN order_items AS oi
    ON o.order_id = oi.order_item_order_id
WHERE o.order_status IN ('COMPLETE', 'CLOSED')
GROUP BY o.order_date;

In [7]:
%%sql
SELECT * FROM daily_revenue
ORDER BY order_date;

,order_date,order_revenue
0,2013-07-25 00:00:00.000000,31547.23
1,2013-07-26 00:00:00.000000,54713.23
2,2013-07-27 00:00:00.000000,48411.48
3,2013-07-28 00:00:00.000000,35672.03
4,2013-07-29 00:00:00.000000,54579.70
...,...,...
359,2014-07-20 00:00:00.000000,60047.45
360,2014-07-21 00:00:00.000000,51427.70
361,2014-07-22 00:00:00.000000,36717.24
362,2014-07-23 00:00:00.000000,38795.23


In [ ]:
%%sql
CREATE TABLE daily_product_revenue
AS
SELECT o.order_date,
    oi.order_item_product_id,
    round(sum(oi.order_item_subtotal)::numeric, 2) AS order_revenue
FROM orders AS o
    JOIN order_items AS oi
        ON o.order_id = oi.order_item_order_id
WHERE o.order_status IN ('COMPLETE', 'CLOSED')
GROUP BY 1, 2;

In [9]:
%%sql
SELECT * FROM daily_product_revenue
ORDER BY 1, 3 DESC;

,order_date,order_item_product_id,order_revenue
0,2013-07-25 00:00:00.000000,1004,5599.72
1,2013-07-25 00:00:00.000000,191,5099.49
2,2013-07-25 00:00:00.000000,957,4499.70
3,2013-07-25 00:00:00.000000,365,3359.44
4,2013-07-25 00:00:00.000000,1073,2999.85
...,...,...,...
9115,2014-07-24 00:00:00.000000,797,35.98
9116,2014-07-24 00:00:00.000000,926,31.98
9117,2014-07-24 00:00:00.000000,172,30.00
9118,2014-07-24 00:00:00.000000,905,24.99


# 54. Overview of OVER and PARTITION BY Clause in SQL Queries

In [15]:
%%sql
-- SELECT * FROM daily_revenue
-- ORDER BY 1;

SELECT to_char(dr.order_date::timestamp, 'yyyy-MM') AS order_month,
    sum(order_revenue) AS order_revenue
FROM daily_revenue AS dr
GROUP BY 1
ORDER BY 1;

,order_month,order_revenue
0,2013-07,333465.45
1,2013-08,1221828.90
2,2013-09,1302255.80
3,2013-10,1171686.92
4,2013-11,1379935.33
5,2013-12,1277719.60
6,2014-01,1230221.74
7,2014-02,1217770.09
8,2014-03,1271350.97
9,2014-04,1249723.52


In [20]:
%%sql
SELECT to_char(dr.order_date::timestamp, 'yyyy-MM') AS order_month,
    dr.order_date,
    dr.order_revenue,
    sum(order_revenue) OVER (
        PARTITION BY to_char(dr.order_date::timestamp, 'yyyy-MM')) AS monthly_order_revenue
FROM daily_revenue AS dr
ORDER BY 1;

,order_month,order_date,order_revenue,monthly_order_revenue
0,2013-07,2013-07-25 00:00:00.000000,31547.23,333465.45
1,2013-07,2013-07-29 00:00:00.000000,54579.70,333465.45
2,2013-07,2013-07-31 00:00:00.000000,59212.49,333465.45
3,2013-07,2013-07-26 00:00:00.000000,54713.23,333465.45
4,2013-07,2013-07-30 00:00:00.000000,49329.29,333465.45
...,...,...,...,...
359,2014-07,2014-07-12 00:00:00.000000,38449.77,955590.77
360,2014-07,2014-07-06 00:00:00.000000,16451.76,955590.77
361,2014-07,2014-07-24 00:00:00.000000,50885.19,955590.77
362,2014-07,2014-07-19 00:00:00.000000,38420.99,955590.77


# 55. Compute Total Aggregation using OVER and PARTITION BY in SQL Queries

In [21]:
%%sql
SELECT dr.*,
       sum(order_revenue) OVER (PARTITION BY 1) as total_revenue
FROM daily_revenue AS dr
ORDER BY 1;

,order_date,order_revenue,total_revenue
0,2013-07-25 00:00:00.000000,31547.23,15012982.48
1,2013-07-26 00:00:00.000000,54713.23,15012982.48
2,2013-07-27 00:00:00.000000,48411.48,15012982.48
3,2013-07-28 00:00:00.000000,35672.03,15012982.48
4,2013-07-29 00:00:00.000000,54579.70,15012982.48
...,...,...,...
359,2014-07-20 00:00:00.000000,60047.45,15012982.48
360,2014-07-21 00:00:00.000000,51427.70,15012982.48
361,2014-07-22 00:00:00.000000,36717.24,15012982.48
362,2014-07-23 00:00:00.000000,38795.23,15012982.48


# 56. Overview of Ranking in SQL
# 57. Compute Global Ranks using SQL

In [23]:
%%sql
-- SELECT count(*) FROM daily_product_revenue;

SELECT * FROM daily_product_revenue
ORDER BY order_date, order_revenue DESC;

,order_date,order_item_product_id,order_revenue
0,2013-07-25 00:00:00.000000,1004,5599.72
1,2013-07-25 00:00:00.000000,191,5099.49
2,2013-07-25 00:00:00.000000,957,4499.70
3,2013-07-25 00:00:00.000000,365,3359.44
4,2013-07-25 00:00:00.000000,1073,2999.85
...,...,...,...
9115,2014-07-24 00:00:00.000000,797,35.98
9116,2014-07-24 00:00:00.000000,926,31.98
9117,2014-07-24 00:00:00.000000,172,30.00
9118,2014-07-24 00:00:00.000000,905,24.99


In [28]:
%%sql
SELECT order_date,
        order_item_product_id,
        order_revenue,
        rank() OVER (ORDER BY order_revenue) AS rnk,
        dense_rank() OVER (ORDER BY order_revenue DESC) AS drnk
FROM daily_product_revenue
WHERE order_date = '2014-01-01 00:00:00.0'
ORDER BY order_revenue DESC;

,order_date,order_item_product_id,order_revenue,rnk,drnk
0,2014-01-01 00:00:00.000000,1004,5599.72,21,1
1,2014-01-01 00:00:00.000000,191,4399.56,20,2
2,2014-01-01 00:00:00.000000,365,3839.36,19,3
3,2014-01-01 00:00:00.000000,502,3300.00,18,4
4,2014-01-01 00:00:00.000000,957,3299.78,17,5
5,2014-01-01 00:00:00.000000,1073,2199.89,16,6
6,2014-01-01 00:00:00.000000,1014,1999.20,15,7
7,2014-01-01 00:00:00.000000,403,1819.86,14,8
8,2014-01-01 00:00:00.000000,627,759.81,13,9
9,2014-01-01 00:00:00.000000,724,500.00,12,10


# 58. Compute Ranks based on key using SQL

In [5]:
%%sql
SELECT order_date,
        order_item_product_id,
        order_revenue,
        rank() OVER (
            PARTITION BY order_date
            ORDER BY order_revenue DESC) AS rnk,
        dense_rank() OVER (
            PARTITION BY order_revenue
            ORDER BY order_revenue DESC) AS drnk
FROM daily_product_revenue
WHERE to_char(order_date, 'yyyy-MM') = '2014-01'
ORDER BY order_revenue DESC;

,order_date,order_item_product_id,order_revenue,rnk,drnk
0,2014-01-30 00:00:00.000000,1004,14799.26,1,1
1,2014-01-11 00:00:00.000000,1004,13599.32,1,1
2,2014-01-22 00:00:00.000000,1004,11999.40,1,1
3,2014-01-05 00:00:00.000000,1004,11999.40,1,1
4,2014-01-03 00:00:00.000000,1004,11599.42,1,1
...,...,...,...,...,...
768,2014-01-31 00:00:00.000000,792,14.99,29,1
769,2014-01-03 00:00:00.000000,792,14.99,30,1
770,2014-01-26 00:00:00.000000,792,14.99,19,1
771,2014-01-28 00:00:00.000000,793,14.99,27,1


# 59. Rules and Restrictions to Filter Data based on Ranks in SQL
# 60. Filtering based on Global Ranks using Nested Queries and CTEs in SQL

In [7]:
%%sql
SELECT * FROM (
SELECT order_date,
        order_item_product_id,
        order_revenue,
        rank() OVER (ORDER BY order_revenue) AS rnk,
        dense_rank() OVER (ORDER BY order_revenue DESC) AS drnk
FROM daily_product_revenue
WHERE order_date = '2014-01-01 00:00:00.0'
ORDER BY order_revenue DESC
) AS q
WHERE drnk <= 5
ORDER BY order_revenue DESC
;

,order_date,order_item_product_id,order_revenue,rnk,drnk
0,2014-01-01 00:00:00.000000,1004,5599.72,21,1
1,2014-01-01 00:00:00.000000,191,4399.56,20,2
2,2014-01-01 00:00:00.000000,365,3839.36,19,3
3,2014-01-01 00:00:00.000000,502,3300.00,18,4
4,2014-01-01 00:00:00.000000,957,3299.78,17,5


In [6]:
%%sql
WITH daily_product_revenue_ranks AS(
    SELECT order_date,
        order_item_product_id,
        order_revenue,
        rank() OVER (ORDER BY order_revenue DESC) AS rnk,
        dense_rank() OVER (ORDER BY order_revenue DESC) AS drnk
FROM daily_product_revenue
WHERE order_date = '2014-01-01 00:00:00.0'
ORDER BY order_revenue DESC
) SELECT * FROM daily_product_revenue_ranks
WHERE drnk <= 5
ORDER BY order_revenue DESC;

,order_date,order_item_product_id,order_revenue,rnk,drnk
0,2014-01-01 00:00:00.000000,1004,5599.72,1,1
1,2014-01-01 00:00:00.000000,191,4399.56,2,2
2,2014-01-01 00:00:00.000000,365,3839.36,3,3
3,2014-01-01 00:00:00.000000,502,3300.00,4,4
4,2014-01-01 00:00:00.000000,957,3299.78,5,5


# 61. Filtering based on Ranks per Partition using Nested Queries and CTEs in SQL

In [12]:
%%sql
SELECT * FROM (
SELECT order_date,
    order_item_product_id,
    order_revenue,
    rank() OVER (
        PARTITION BY order_date
        ORDER BY order_revenue DESC
    ) AS rnk,
    dense_rank() OVER (
        PARTITION BY order_date
        ORDER BY order_revenue DESC
    ) AS drnk
FROM daily_product_revenue
WHERE to_char(order_date::date, 'yyyy-MM') = '2014-01'
) AS q
WHERE drnk <= 5
ORDER BY order_date, order_revenue DESC
;

,order_date,order_item_product_id,order_revenue,rnk,drnk
0,2014-01-01 00:00:00.000000,1004,5599.72,1,1
1,2014-01-01 00:00:00.000000,191,4399.56,2,2
2,2014-01-01 00:00:00.000000,365,3839.36,3,3
3,2014-01-01 00:00:00.000000,502,3300.00,4,4
4,2014-01-01 00:00:00.000000,957,3299.78,5,5
...,...,...,...,...,...
151,2014-01-31 00:00:00.000000,1004,5999.70,1,1
152,2014-01-31 00:00:00.000000,502,4950.00,2,2
153,2014-01-31 00:00:00.000000,403,4809.63,3,3
154,2014-01-31 00:00:00.000000,1073,4799.76,4,4


In [16]:
%%sql
WITH daily_product_revenue_ranks AS (
    SELECT order_date,
    order_item_product_id,
    order_revenue,
    rank() OVER (
        PARTITION BY order_date
        ORDER BY order_revenue DESC
    ) AS rnk,
    dense_rank() OVER (
        PARTITION BY order_date
        ORDER BY order_revenue DESC
    ) AS drnk
FROM daily_product_revenue
WHERE to_char(order_date::date, 'yyyy-MM') = '2014-01'
) SELECT * FROM daily_product_revenue_ranks
WHERE drnk <= 5
ORDER BY order_date, order_revenue DESC;

,order_date,order_item_product_id,order_revenue,rnk,drnk
0,2014-01-01 00:00:00.000000,1004,5599.72,1,1
1,2014-01-01 00:00:00.000000,191,4399.56,2,2
2,2014-01-01 00:00:00.000000,365,3839.36,3,3
3,2014-01-01 00:00:00.000000,502,3300.00,4,4
4,2014-01-01 00:00:00.000000,957,3299.78,5,5
...,...,...,...,...,...
151,2014-01-31 00:00:00.000000,1004,5999.70,1,1
152,2014-01-31 00:00:00.000000,502,4950.00,2,2
153,2014-01-31 00:00:00.000000,403,4809.63,3,3
154,2014-01-31 00:00:00.000000,1073,4799.76,4,4


# 62. Create Students table with Data for ranking using SQL